In [30]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from uncertainties import ufloat
from uncertainties.umath import *  # sin(), etc.
import uncertainties.unumpy as un

df_placa = pd.read_excel('D:/UNESP/EngMec-UNESP/Lab. Mec Flu II/Lab 1 - Medidores de vazão/dados.xlsx')
df_vent = pd.read_excel('D:/UNESP/EngMec-UNESP/Lab. Mec Flu II/Lab 1 - Medidores de vazão/dados.xlsx', sheet_name='venturi')

In [31]:
df_placa

,h1,h2,del,ve
0,18.8,20.7,1.9,3.40
1,16.5,22.9,6.4,5.94
2,14.2,25.2,11.0,7.65
3,12.5,26.8,14.3,8.60
4,10.0,29.8,19.8,9.93


In [32]:
df_vent

,h1,h2,del,ve
0,19.3,20.5,1.2,3.60
1,17.8,22.0,4.2,6.40
2,15.9,24.0,8.1,8.80
3,14.1,26.0,11.9,10.58
4,9.5,30.8,21.3,13.00


In [33]:
rho_ar = 1.20
rho_agua = 997
mi = 1.80e-5
g = 9.81

def calc_vazão(Dt, D1, delta_h, venturi=True):
    
    beta = Dt/D1
    At = np.pi*(Dt**2)/4
    A1 = np.pi*(D1**2)/4

    delta_P = rho_agua*g*(delta_h/100)

    lista_m = []
    for j in range(len(delta_P)):
        m_dot  = 10.0
        for i in range(100):

            v = m_dot/(rho_ar*A1)
            Re = (rho_ar*v*D1)/mi

            if venturi:
                C = 0.99
            else:
                C = 0.5959 + (0.0312*(beta**2.1)) - (0.184*beta**8) + ((91.71*beta**2.5)/(Re**0.75))
                
            K = C/np.sqrt(1 - beta**4)
            m_dot_new = K*At*np.sqrt(2*rho_ar*(delta_P[j]))
            
            if abs(m_dot - m_dot_new) < 1e-8:
                break
            else:
                m_dot = m_dot_new

        lista_m.append(m_dot_new)
        
    lista_m = np.array(lista_m)
    lista_Q = lista_m/rho_ar
    lista_V = lista_Q/A1

    return lista_m, lista_Q, lista_V #kg/s, m^3/s, m/s

In [34]:
#Placa de orificio
placa = calc_vazão(0.0257, 0.0385, df_placa['del'].values, False)
placa

(array([0.00768234, 0.01389186, 0.01812435, 0.02062295, 0.0242119 ]),
 array([0.00640195, 0.01157655, 0.01510362, 0.0171858 , 0.02017659]),
 array([ 5.49921472,  9.94415423, 12.97387612, 14.76244508, 17.33150759]))

In [35]:
res_p = pd.DataFrame({'teo':placa[2], 'exp': df_placa['ve'].values, 
                     'erro':abs(df_placa['ve'].values - placa[2])/placa[2]*100})
res_p

,teo,exp,erro
0,5.499215,3.40,38.172991
1,9.944154,5.94,40.266413
2,12.973876,7.65,41.035355
3,14.762445,8.60,41.744068
4,17.331508,9.93,42.705504


In [36]:
print(res_p.to_latex(index=False))

\begin{tabular}{rrr}
\toprule
      teo &  exp &      erro \\
\midrule
 5.499215 & 3.40 & 38.172991 \\
 9.944154 & 5.94 & 40.266413 \\
12.973876 & 7.65 & 41.035355 \\
14.762445 & 8.60 & 41.744068 \\
17.331508 & 9.93 & 42.705504 \\
\bottomrule
\end{tabular}



In [37]:
#Tubo de venturi
tubo = calc_vazão(0.0244, 0.0385, df_vent['del'].values)
tubo

(array([0.00848375, 0.01587163, 0.02204142, 0.02671594, 0.03574265]),
 array([0.00706979, 0.01322636, 0.01836785, 0.02226328, 0.02978555]),
 array([ 6.0728847 , 11.36132694, 15.77781726, 19.12396293, 25.585518  ]))

In [38]:
res_t = pd.DataFrame({'teo':tubo[2], 'exp': df_vent['ve'].values, 
                     'erro':abs(df_vent['ve'].values - tubo[2])/tubo[2]*100})
res_t

,teo,exp,erro
0,6.072885,3.60,40.720100
1,11.361327,6.40,43.668552
2,15.777817,8.80,44.225492
3,19.123963,10.58,44.676739
4,25.585518,13.00,49.190007


In [39]:
print(res_t.to_latex(index=False))

\begin{tabular}{rrr}
\toprule
      teo &   exp &      erro \\
\midrule
 6.072885 &  3.60 & 40.720100 \\
11.361327 &  6.40 & 43.668552 \\
15.777817 &  8.80 & 44.225492 \\
19.123963 & 10.58 & 44.676739 \\
25.585518 & 13.00 & 49.190007 \\
\bottomrule
\end{tabular}



In [40]:
#ERROS ENTRE AS PLACAS
res_t = pd.DataFrame({'placa':placa[2], 'tubo': tubo[2], 
                     'erro':abs(tubo[2] - placa[2])/placa[2]*100})
res_t

,placa,tubo,erro
0,5.499215,6.072885,10.431853
1,9.944154,11.361327,14.251315
2,12.973876,15.777817,21.612208
3,14.762445,19.123963,29.544685
4,17.331508,25.585518,47.624307


In [41]:
print(res_t.to_latex(index=False))

\begin{tabular}{rrr}
\toprule
    placa &      tubo &      erro \\
\midrule
 5.499215 &  6.072885 & 10.431853 \\
 9.944154 & 11.361327 & 14.251315 \\
12.973876 & 15.777817 & 21.612208 \\
14.762445 & 19.123963 & 29.544685 \\
17.331508 & 25.585518 & 47.624307 \\
\bottomrule
\end{tabular}

